<a href="https://colab.research.google.com/github/sahilsinghiitb/Credit_based_lending_classification/blob/main/Credit_risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here we are going to do a project where we try to calculate risk whether we have to give a customer loan or not.(with respect to bank)

some banking term:

Asset- loan product(housing and personal loan) , something which gives profit to bank, Credit card

Liability:Current account, savings account, fixed deposit, RD

CASA(current and saving), Term deposit(RD,FD)

Dibursed Amount- Loan amount given to a customer

OSP- Outstanding principle(let say 10000 lended as a loan and i payed 1000 then osp is 9000 now, at the end it become zero)

Amortization-

DPD- Days past due (days uhh skipped your emi's,'defaulted')

PAR-Portfolio at risk[Osp when DPD > 0]

NPA- (non performing asset){loan that is defaulted}
loan Account, when DPD > 90

**Credit risk types in banking:**
DPD= 0 (NDA- Non delinquint Account)

DPD= 0 to 30 (SMA1- Standard monitoring Account 1)

DPD= 31 to 60 (SMA2- Standard monitoring Account 2)

DPD= 60 to 90 SMA3

DPD= 90 to 180 NPA

DPD>180: Written off (loan which is not present) loan portfolio quality of the bank will be better= market sentiment will be better= increase stock price = good book size= if is on sensex top 30, there it will improve

GNPA- (3 - 5 % )= OSP default

NNPA- net npa (0.01 - 0.06 %)= Provisioning Amount Subtracted

Bank quality assess, GNPA value

case study 2: CIBIL Dataset

Response- P1, P2, P3, P4  Priority level(multiclass classification)

Case study 1: Internal data

loan- Tradeline, secured loan- collatoral based, Deliquency- Default

In [ ]:
#Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
import warnings
import os

In [ ]:
#read the dataset
a1=pd.read_excel("/content/case_study1 (1).xlsx")
a2=pd.read_excel('/content/case_study2.xlsx')

In [ ]:
df1=a1.copy()
df2=a2.copy()

In [ ]:
print(df1.head(5))
print(df2.head(5))

   PROSPECTID  Total_TL  Tot_Closed_TL  Tot_Active_TL  Total_TL_opened_L6M  \
0           1         5              4              1                    0   
1           2         1              0              1                    0   
2           3         8              0              8                    1   
3           4         1              0              1                    1   
4           5         3              2              1                    0   

   Tot_TL_closed_L6M  pct_tl_open_L6M  pct_tl_closed_L6M  pct_active_tl  \
0                  0            0.000                0.0          0.200   
1                  0            0.000                0.0          1.000   
2                  0            0.125                0.0          1.000   
3                  0            1.000                0.0          1.000   
4                  0            0.000                0.0          0.333   

   pct_closed_tl  ...  CC_TL  Consumer_TL  Gold_TL  Home_TL  PL_TL  \
0         

In [ ]:
print(df1.info())
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51336 entries, 0 to 51335
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PROSPECTID            51336 non-null  int64  
 1   Total_TL              51336 non-null  int64  
 2   Tot_Closed_TL         51336 non-null  int64  
 3   Tot_Active_TL         51336 non-null  int64  
 4   Total_TL_opened_L6M   51336 non-null  int64  
 5   Tot_TL_closed_L6M     51336 non-null  int64  
 6   pct_tl_open_L6M       51336 non-null  float64
 7   pct_tl_closed_L6M     51336 non-null  float64
 8   pct_active_tl         51336 non-null  float64
 9   pct_closed_tl         51336 non-null  float64
 10  Total_TL_opened_L12M  51336 non-null  int64  
 11  Tot_TL_closed_L12M    51336 non-null  int64  
 12  pct_tl_open_L12M      51336 non-null  float64
 13  pct_tl_closed_L12M    51336 non-null  float64
 14  Tot_Missed_Pmnt       51336 non-null  int64  
 15  Auto_TL            

In [ ]:
print(df1.describe())
print(df2.describe())

         PROSPECTID      Total_TL  Tot_Closed_TL  Tot_Active_TL  \
count  51336.000000  51336.000000   51336.000000   51336.000000   
mean   25668.500000      4.858598       2.770415       2.088184   
std    14819.571046      7.177116       5.941680       2.290774   
min        1.000000      1.000000       0.000000       0.000000   
25%    12834.750000      1.000000       0.000000       1.000000   
50%    25668.500000      2.000000       1.000000       1.000000   
75%    38502.250000      5.000000       3.000000       3.000000   
max    51336.000000    235.000000     216.000000      47.000000   

       Total_TL_opened_L6M  Tot_TL_closed_L6M  pct_tl_open_L6M  \
count         51336.000000       51336.000000     51336.000000   
mean              0.736754           0.428919         0.184574   
std               1.296717           0.989972         0.297414   
min               0.000000           0.000000         0.000000   
25%               0.000000           0.000000         0.000000   


In [ ]:
#remove null(since it is of very small fraction)
df1=df1.loc[df1['Age_Oldest_TL']!= -99999]

In [ ]:
df1.describe()

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
count,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,...,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000
mean,25668.574353,4.861471,2.772321,2.089149,0.737328,0.429254,0.184718,0.089164,0.577349,0.422651,...,0.125078,1.136970,1.563046,0.070181,0.282712,2.846752,2.014718,1.089773,45.376969,15.780022
std,14819.611579,7.179141,5.943536,2.291375,1.297059,0.990286,0.297486,0.205700,0.379804,0.379804,...,0.505386,2.228639,5.378357,0.340963,0.858461,6.189165,3.199333,2.418314,41.737151,21.995676
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,12836.750000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,4.000000
50%,25666.500000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.556000,0.444000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,33.000000,8.000000
75%,38503.250000,5.000000,3.000000,3.000000,1.000000,1.000000,0.312000,0.054000,1.000000,0.750000,...,0.000000,1.000000,1.000000,0.000000,0.000000,3.000000,2.000000,1.000000,64.000000,17.000000
max,51336.000000,235.000000,216.000000,47.000000,27.000000,19.000000,1.000000,1.000000,1.000000,1.000000,...,27.000000,41.000000,235.000000,10.000000,29.000000,235.000000,55.000000,80.000000,392.000000,392.000000


In [ ]:
columns_to_beremoved=[]
for i in df2.columns:
  if sum(df2.loc[:,i]==-99999)>10000:
    columns_to_beremoved.append(i)

print(columns_to_beremoved)

['time_since_first_deliquency', 'time_since_recent_deliquency', 'max_delinquency_level', 'max_deliq_6mts', 'max_deliq_12mts', 'CC_utilization', 'PL_utilization', 'max_unsec_exposure_inPct']


In [ ]:
#removing these column because it has null values >10000
df2=df2.drop(columns=columns_to_beremoved)

In [ ]:
for i in df2.columns:
  df2= df2.loc[df2[i]!=-99999]

In [ ]:
df2.shape

(42066, 54)

In [ ]:
np.sum(df2.isnull())

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


,0
PROSPECTID,0
time_since_recent_payment,0
num_times_delinquent,0
max_recent_level_of_deliq,0
num_deliq_6mts,0
num_deliq_12mts,0
num_deliq_6_12mts,0
num_times_30p_dpd,0
num_times_60p_dpd,0
num_std,0


In [ ]:
#common column is prospect_ID
df=pd.merge(df1,df2,left_on=['PROSPECTID'],right_on=['PROSPECTID'],how='inner')

In [ ]:
df.shape

(42064, 79)

In [ ]:
df.isna().sum()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 79 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PROSPECTID                  42064 non-null  int64  
 1   Total_TL                    42064 non-null  int64  
 2   Tot_Closed_TL               42064 non-null  int64  
 3   Tot_Active_TL               42064 non-null  int64  
 4   Total_TL_opened_L6M         42064 non-null  int64  
 5   Tot_TL_closed_L6M           42064 non-null  int64  
 6   pct_tl_open_L6M             42064 non-null  float64
 7   pct_tl_closed_L6M           42064 non-null  float64
 8   pct_active_tl               42064 non-null  float64
 9   pct_closed_tl               42064 non-null  float64
 10  Total_TL_opened_L12M        42064 non-null  int64  
 11  Tot_TL_closed_L12M          42064 non-null  int64  
 12  pct_tl_open_L12M            42064 non-null  float64
 13  pct_tl_closed_L12M          420

In [ ]:
#categorical features
column_categorical=[]
for i in df.columns:
  if df[i].dtype=='object':
    column_categorical.append(i)
print(column_categorical)

['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2', 'Approved_Flag']


In [ ]:
#Hypothesis testing
#H0: Thre is no association between Marital station and Approved Flag
#H1:There is association
#Chiisquare Test

# Variables
predictors = ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']
response = 'Approved_Flag'

# Loop through each predictor and perform Chi-square test
for predictor in predictors:
    print(f"\nChi-Square Test between {predictor} and {response}:")

    # Create contingency table
    contingency_table = pd.crosstab(df[predictor], df[response])

    # Perform Chi-square test
    chi2, p, dof, expected = chi2_contingency(contingency_table)

    # Output results
    print("Chi2 Statistic:", chi2)
    print("Degrees of Freedom:", dof)
    print("P-value:", p)

    if p < 0.05:
        print("=> Significant association (reject H0)")
    else:
        print("=> No significant association (fail to reject H0)")



Chi-Square Test between MARITALSTATUS and Approved_Flag:
Chi2 Statistic: 1076.9871387543772
Degrees of Freedom: 3
P-value: 3.578180861038862e-233
=> Significant association (reject H0)

Chi-Square Test between EDUCATION and Approved_Flag:
Chi2 Statistic: 187.81675366240617
Degrees of Freedom: 18
P-value: 2.6942265249737532e-30
=> Significant association (reject H0)

Chi-Square Test between GENDER and Approved_Flag:
Chi2 Statistic: 24.56031272141628
Degrees of Freedom: 3
P-value: 1.907936100186563e-05
=> Significant association (reject H0)

Chi-Square Test between last_prod_enq2 and Approved_Flag:
Chi2 Statistic: 2444.9571510235596
Degrees of Freedom: 15
P-value: 0.0
=> Significant association (reject H0)

Chi-Square Test between first_prod_enq2 and Approved_Flag:
Chi2 Statistic: 1387.5609151031795
Degrees of Freedom: 15
P-value: 7.84997610555419e-287
=> Significant association (reject H0)


We accept all the predictors in model

In [ ]:
numerical_column=[]
for i in df.columns:
  if df[i].dtype!='object' and i not in ['PROSPECTID','Approved_Flag']:
    numerical_column.append(i)
print(numerical_column)

['Total_TL', 'Tot_Closed_TL', 'Tot_Active_TL', 'Total_TL_opened_L6M', 'Tot_TL_closed_L6M', 'pct_tl_open_L6M', 'pct_tl_closed_L6M', 'pct_active_tl', 'pct_closed_tl', 'Total_TL_opened_L12M', 'Tot_TL_closed_L12M', 'pct_tl_open_L12M', 'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'Auto_TL', 'CC_TL', 'Consumer_TL', 'Gold_TL', 'Home_TL', 'PL_TL', 'Secured_TL', 'Unsecured_TL', 'Other_TL', 'Age_Oldest_TL', 'Age_Newest_TL', 'time_since_recent_payment', 'num_times_delinquent', 'max_recent_level_of_deliq', 'num_deliq_6mts', 'num_deliq_12mts', 'num_deliq_6_12mts', 'num_times_30p_dpd', 'num_times_60p_dpd', 'num_std', 'num_std_6mts', 'num_std_12mts', 'num_sub', 'num_sub_6mts', 'num_sub_12mts', 'num_dbt', 'num_dbt_6mts', 'num_dbt_12mts', 'num_lss', 'num_lss_6mts', 'num_lss_12mts', 'recent_level_of_deliq', 'tot_enq', 'CC_enq', 'CC_enq_L6m', 'CC_enq_L12m', 'PL_enq', 'PL_enq_L6m', 'PL_enq_L12m', 'time_since_recent_enq', 'enq_L12m', 'enq_L6m', 'enq_L3m', 'AGE', 'NETMONTHLYINCOME', 'Time_With_Curr_Empr', 'pct_

Sequential VIF

In [ ]:
len(numerical_column)

72

In [ ]:
vif_data=df[numerical_column]
total_columns=vif_data.shape[1]
columns_to_be_kept=[]
column_index=0

for i in range(0,total_columns):
  vif_value=variance_inflation_factor(vif_data,column_index)
  print(column_index,"---", vif_value)
  if vif_value<=6:
    columns_to_be_kept.append(numerical_column[i])
    column_index=column_index+1
  else:
    vif_data=vif_data.drop([numerical_column[i]], axis=1)
print(columns_to_be_kept)

/usr/local/lib/python3.11/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf


/usr/local/lib/python3.11/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf
0 --- 11.320180023967996
0 --- 8.363698035000327
0 --- 6.520647877790928
0 --- 5.149501618212625
1 --- 2.611111040579735


/usr/local/lib/python3.11/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


2 --- inf
2 --- 1788.7926256209232
2 --- 8.601028256477228
2 --- 3.8328007921530785
3 --- 6.099653381646723
3 --- 5.581352009642766
4 --- 1.9855843530987776


/usr/local/lib/python3.11/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


5 --- inf
5 --- 4.809538302819343
6 --- 23.270628983464636
6 --- 30.595522588100053
6 --- 4.3843464059655854
7 --- 3.064658415523423
8 --- 2.898639771299253
9 --- 4.377876915347322
10 --- 2.207853583695844
11 --- 4.916914200506861
12 --- 5.214702030064725
13 --- 3.3861625024231476
14 --- 7.840583309478997
14 --- 5.255034641721438


/usr/local/lib/python3.11/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


15 --- inf
15 --- 7.380634506427238
15 --- 1.4210050015175735
16 --- 8.083255010190316
16 --- 1.624122752404011
17 --- 7.257811920140003
17 --- 15.59624383268298
17 --- 1.8258570471324318
18 --- 1.5080839450032664
19 --- 2.1720888348245753
20 --- 2.623397553527229
21 --- 2.2959970812106167
22 --- 7.360578319196439
22 --- 2.1602387773102554
23 --- 2.8686288267891444
24 --- 6.458218003637272
24 --- 2.847411886563824
25 --- 4.75319815628408
26 --- 16.22735475594825
26 --- 6.424377256363877
26 --- 8.887080381808678
26 --- 2.3804746142952666
27 --- 8.609513476514548
27 --- 13.06755093547673
27 --- 3.500040056654653
28 --- 1.9087955874813773
29 --- 17.006562234161628
29 --- 10.730485153719197
29 --- 2.3538497522950275
30 --- 22.104855915136433
30 --- 2.7971639638512915
31 --- 3.424171203217697
32 --- 10.175021454450922
32 --- 6.408710354561292
32 --- 1.001151196262563
33 --- 3.069197305397274
34 --- 2.8091261600643707
35 --- 20.249538381980678
35 --- 15.864576541593745
35 --- 1.8331649740532

In [ ]:
len(columns_to_be_kept)

39

In [ ]:
# check Anova for columns_to_be_kept

from scipy.stats import f_oneway

columns_to_be_kept_numerical = []

for i in columns_to_be_kept:
    a = list(df[i])
    b = list(df['Approved_Flag'])

    group_P1 = [value for value, group in zip(a, b) if group == 'P1']
    group_P2 = [value for value, group in zip(a, b) if group == 'P2']
    group_P3 = [value for value, group in zip(a, b) if group == 'P3']
    group_P4 = [value for value, group in zip(a, b) if group == 'P4']


    f_statistic, p_value = f_oneway(group_P1, group_P2, group_P3, group_P4)

    if p_value <= 0.05:
        columns_to_be_kept_numerical.append(i)

In [ ]:
columns_to_be_kept_numerical

['pct_tl_open_L6M',
 'pct_tl_closed_L6M',
 'Tot_TL_closed_L12M',
 'pct_tl_closed_L12M',
 'Tot_Missed_Pmnt',
 'CC_TL',
 'Home_TL',
 'PL_TL',
 'Secured_TL',
 'Unsecured_TL',
 'Other_TL',
 'Age_Oldest_TL',
 'Age_Newest_TL',
 'time_since_recent_payment',
 'max_recent_level_of_deliq',
 'num_deliq_6_12mts',
 'num_times_60p_dpd',
 'num_std_12mts',
 'num_sub',
 'num_sub_6mts',
 'num_sub_12mts',
 'num_dbt',
 'num_dbt_12mts',
 'num_lss',
 'recent_level_of_deliq',
 'CC_enq_L12m',
 'PL_enq_L12m',
 'time_since_recent_enq',
 'enq_L3m',
 'NETMONTHLYINCOME',
 'Time_With_Curr_Empr',
 'CC_Flag',
 'PL_Flag',
 'pct_PL_enq_L6m_of_ever',
 'pct_CC_enq_L6m_of_ever',
 'HL_Flag',
 'GL_Flag']

In [ ]:
len(columns_to_be_kept_numerical)

37

In [ ]:
df

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,5,4,1,0,0,0.000,0.00,0.200,0.800,...,0.0,0.0,0.000,0.0,1,0,PL,PL,696,P2
1,2,1,0,1,0,0,0.000,0.00,1.000,0.000,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,8,0,8,1,0,0.125,0.00,1.000,0.000,...,0.0,0.0,0.000,0.0,1,0,ConsumerLoan,others,693,P2
3,5,3,2,1,0,0,0.000,0.00,0.333,0.667,...,0.0,0.0,0.000,0.0,0,0,AL,AL,753,P1
4,6,6,5,1,0,0,0.000,0.00,0.167,0.833,...,1.0,0.0,0.429,0.0,1,0,ConsumerLoan,PL,668,P3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42059,51332,3,0,3,1,0,0.333,0.00,1.000,0.000,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,650,P4
42060,51333,4,2,2,0,1,0.000,0.25,0.500,0.500,...,0.0,0.0,0.000,0.0,0,0,others,others,702,P1
42061,51334,2,1,1,1,1,0.500,0.50,0.500,0.500,...,1.0,0.0,1.000,0.0,0,0,ConsumerLoan,others,661,P3
42062,51335,2,1,1,0,0,0.000,0.00,0.500,0.500,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,others,686,P2


In [ ]:
list1 = columns_to_be_kept  # your list with 37 objects
list2 = columns_to_be_kept_numerical  # your list with 39 objects

# Convert to sets for fast comparison
set1 = set(list1)
set2 = set(list2)

# Common elements
common = set1 & set2  # or set1.intersection(set2)

# Unique to list1
only_in_list1 = set1 - set2

# Unique to list2
only_in_list2 = set2 - set1

print("Common elements:", common)
print("Only in list1:", only_in_list1)
print("Only in list2:", only_in_list2)

Common elements: {'Age_Newest_TL', 'pct_PL_enq_L6m_of_ever', 'CC_TL', 'Age_Oldest_TL', 'GL_Flag', 'Home_TL', 'PL_TL', 'time_since_recent_payment', 'recent_level_of_deliq', 'num_sub', 'Secured_TL', 'pct_CC_enq_L6m_of_ever', 'num_lss', 'num_dbt', 'pct_tl_closed_L6M', 'Unsecured_TL', 'enq_L3m', 'CC_enq_L12m', 'num_times_60p_dpd', 'pct_tl_open_L6M', 'Tot_TL_closed_L12M', 'HL_Flag', 'Other_TL', 'num_dbt_12mts', 'NETMONTHLYINCOME', 'num_sub_6mts', 'CC_Flag', 'Tot_Missed_Pmnt', 'PL_enq_L12m', 'num_sub_12mts', 'pct_tl_closed_L12M', 'PL_Flag', 'time_since_recent_enq', 'max_recent_level_of_deliq', 'Time_With_Curr_Empr', 'num_deliq_6_12mts', 'num_std_12mts'}
Only in list1: {'pct_currentBal_all_TL', 'num_lss_12mts'}
Only in list2: set()


In [ ]:
print(df['MARITALSTATUS'].unique())
print(df['EDUCATION'].unique(),
df['GENDER'].unique(),
df['last_prod_enq2'].unique(),
df['first_prod_enq2'].unique())

['Married' 'Single']
['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS'
 'PROFESSIONAL'] ['M' 'F'] ['PL' 'ConsumerLoan' 'AL' 'CC' 'others' 'HL'] ['PL' 'ConsumerLoan' 'others' 'AL' 'HL' 'CC']


In [ ]:
#ordinal features
df.loc[df['EDUCATION']=='SSC',['EDUCATION']]=1
df.loc[df['EDUCATION']=='12TH',['EDUCATION']]=2
df.loc[df['EDUCATION']=='GRADUATE',['EDUCATION']]=3
df.loc[df['EDUCATION']=='UNDER GRADUATE',['EDUCATION']]=3
df.loc[df['EDUCATION']=='POST-GRADUATE',['EDUCATION']]=4
df.loc[df['EDUCATION']=='OTHERS',['EDUCATION']]=1
df.loc[df['EDUCATION']=='PROFESSIONAL',['EDUCATION']]=3

In [ ]:
column_to_be_in_final = list(set(columns_to_be_kept_numerical + column_categorical))


In [ ]:
column_to_be_in_final

['Age_Newest_TL',
 'pct_PL_enq_L6m_of_ever',
 'CC_TL',
 'Age_Oldest_TL',
 'GL_Flag',
 'Home_TL',
 'EDUCATION',
 'PL_TL',
 'time_since_recent_payment',
 'last_prod_enq2',
 'recent_level_of_deliq',
 'num_sub',
 'Secured_TL',
 'pct_CC_enq_L6m_of_ever',
 'num_lss',
 'num_dbt',
 'pct_tl_closed_L6M',
 'Unsecured_TL',
 'enq_L3m',
 'CC_enq_L12m',
 'num_times_60p_dpd',
 'pct_tl_open_L6M',
 'Tot_TL_closed_L12M',
 'HL_Flag',
 'Other_TL',
 'num_dbt_12mts',
 'first_prod_enq2',
 'NETMONTHLYINCOME',
 'num_sub_6mts',
 'CC_Flag',
 'Tot_Missed_Pmnt',
 'PL_enq_L12m',
 'GENDER',
 'num_sub_12mts',
 'pct_tl_closed_L12M',
 'PL_Flag',
 'time_since_recent_enq',
 'max_recent_level_of_deliq',
 'Time_With_Curr_Empr',
 'Approved_Flag',
 'num_deliq_6_12mts',
 'MARITALSTATUS',
 'num_std_12mts']

In [ ]:
df.loc[:,'EDUCATION']

,EDUCATION
0,2
1,3
2,1
3,4
4,2
...,...
42059,2
42060,1
42061,1
42062,3


In [ ]:
df1=df.loc[:,column_to_be_in_final]
df1

,Age_Newest_TL,pct_PL_enq_L6m_of_ever,CC_TL,Age_Oldest_TL,GL_Flag,Home_TL,EDUCATION,PL_TL,time_since_recent_payment,last_prod_enq2,...,num_sub_12mts,pct_tl_closed_L12M,PL_Flag,time_since_recent_enq,max_recent_level_of_deliq,Time_With_Curr_Empr,Approved_Flag,num_deliq_6_12mts,MARITALSTATUS,num_std_12mts
0,18,0.000,0,72,0,0,2,4,549,PL,...,0,0.000,1,566,29,114,P2,0,Married,11
1,7,0.000,0,7,0,0,3,0,47,ConsumerLoan,...,0,0.000,0,209,0,50,P2,0,Single,0
2,2,0.000,0,47,0,0,1,0,302,ConsumerLoan,...,0,0.000,0,587,25,191,P2,8,Married,10
3,32,0.000,0,131,0,0,4,0,583,AL,...,0,0.000,0,3951,0,75,P1,0,Married,16
4,17,0.429,0,150,0,0,2,0,245,ConsumerLoan,...,1,0.167,0,7,270,154,P3,0,Married,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42059,5,0.000,0,24,0,0,2,0,15,ConsumerLoan,...,0,0.000,0,0,24,249,P4,0,Married,0
42060,7,0.000,0,74,0,0,1,0,57,others,...,0,0.250,0,203,0,186,P1,0,Married,6
42061,5,1.000,0,9,0,0,1,0,32,ConsumerLoan,...,0,0.500,0,1,0,66,P3,0,Married,0
42062,8,0.000,0,15,0,0,3,0,58,ConsumerLoan,...,0,0.500,0,242,0,54,P2,0,Single,0


In [ ]:
df_encoded=pd.get_dummies(df1,columns=['MARITALSTATUS','GENDER','last_prod_enq2','first_prod_enq2'])

In [ ]:
df_encoded

,Age_Newest_TL,pct_PL_enq_L6m_of_ever,CC_TL,Age_Oldest_TL,GL_Flag,Home_TL,EDUCATION,PL_TL,time_since_recent_payment,recent_level_of_deliq,...,last_prod_enq2_ConsumerLoan,last_prod_enq2_HL,last_prod_enq2_PL,last_prod_enq2_others,first_prod_enq2_AL,first_prod_enq2_CC,first_prod_enq2_ConsumerLoan,first_prod_enq2_HL,first_prod_enq2_PL,first_prod_enq2_others
0,18,0.000,0,72,0,0,2,4,549,29,...,False,False,True,False,False,False,False,False,True,False
1,7,0.000,0,7,0,0,3,0,47,0,...,True,False,False,False,False,False,True,False,False,False
2,2,0.000,0,47,0,0,1,0,302,25,...,True,False,False,False,False,False,False,False,False,True
3,32,0.000,0,131,0,0,4,0,583,0,...,False,False,False,False,True,False,False,False,False,False
4,17,0.429,0,150,0,0,2,0,245,26,...,True,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42059,5,0.000,0,24,0,0,2,0,15,24,...,True,False,False,False,False,False,True,False,False,False
42060,7,0.000,0,74,0,0,1,0,57,0,...,False,False,False,True,False,False,False,False,False,True
42061,5,1.000,0,9,0,0,1,0,32,0,...,True,False,False,False,False,False,False,False,False,True
42062,8,0.000,0,15,0,0,3,0,58,0,...,True,False,False,False,False,False,False,False,False,True


In [ ]:
df_encoded['EDUCATION']=df['EDUCATION'].astype(int)

In [ ]:
df_encoded

,Age_Newest_TL,pct_PL_enq_L6m_of_ever,CC_TL,Age_Oldest_TL,GL_Flag,Home_TL,EDUCATION,PL_TL,time_since_recent_payment,recent_level_of_deliq,...,last_prod_enq2_ConsumerLoan,last_prod_enq2_HL,last_prod_enq2_PL,last_prod_enq2_others,first_prod_enq2_AL,first_prod_enq2_CC,first_prod_enq2_ConsumerLoan,first_prod_enq2_HL,first_prod_enq2_PL,first_prod_enq2_others
0,18,0.000,0,72,0,0,2,4,549,29,...,False,False,True,False,False,False,False,False,True,False
1,7,0.000,0,7,0,0,3,0,47,0,...,True,False,False,False,False,False,True,False,False,False
2,2,0.000,0,47,0,0,1,0,302,25,...,True,False,False,False,False,False,False,False,False,True
3,32,0.000,0,131,0,0,4,0,583,0,...,False,False,False,False,True,False,False,False,False,False
4,17,0.429,0,150,0,0,2,0,245,26,...,True,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42059,5,0.000,0,24,0,0,2,0,15,24,...,True,False,False,False,False,False,True,False,False,False
42060,7,0.000,0,74,0,0,1,0,57,0,...,False,False,False,True,False,False,False,False,False,True
42061,5,1.000,0,9,0,0,1,0,32,0,...,True,False,False,False,False,False,False,False,False,True
42062,8,0.000,0,15,0,0,3,0,58,0,...,True,False,False,False,False,False,False,False,False,True


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Create directory if it doesn't exist
import os

save_path = '/content/drive/My Drive/encoded_data'
os.makedirs(save_path, exist_ok=True)

# Save the DataFrame
df_encoded.to_csv(f'{save_path}/df_encoded.csv', index=False)

In [ ]:
df_encoded=pd.read_csv('/content/drive/My Drive/encoded_data/df_encoded.csv')

In [ ]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 55 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Age_Newest_TL                 42064 non-null  int64  
 1   pct_PL_enq_L6m_of_ever        42064 non-null  float64
 2   CC_TL                         42064 non-null  int64  
 3   Age_Oldest_TL                 42064 non-null  int64  
 4   GL_Flag                       42064 non-null  int64  
 5   Home_TL                       42064 non-null  int64  
 6   EDUCATION                     42064 non-null  int64  
 7   PL_TL                         42064 non-null  int64  
 8   time_since_recent_payment     42064 non-null  int64  
 9   recent_level_of_deliq         42064 non-null  int64  
 10  num_sub                       42064 non-null  int64  
 11  Secured_TL                    42064 non-null  int64  
 12  pct_CC_enq_L6m_of_ever        42064 non-null  float64
 13  n

In [ ]:
y=df_encoded['Approved_Flag']
x=df_encoded.drop(['Approved_Flag'],axis=1)

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=200, random_state=42)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200, random_state=42)

In [ ]:
y_pred = rf_classifier.predict(X_test)

In [ ]:
accuracy=accuracy_score(y_test,y_pred)
print(accuracy)

0.7628669915606799


In [ ]:
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

In [ ]:
for i, v in enumerate(['p1','p2','p3','p4']):
    print(f"{v} Precision: {precision[i]}")
    print(f"{v} Recall: {recall[i]}")
    print(f"{v} F1 Score: {f1_score[i]}")

p1 Precision: 0.8362989323843416
p1 Recall: 0.6952662721893491
p1 F1 Score: 0.7592891760904685
p2 Precision: 0.7939681463910538
p2 Recall: 0.9288404360753221
p2 F1 Score: 0.8561249657440395
p3 Precision: 0.440625
p3 Recall: 0.21283018867924527
p3 F1 Score: 0.2870229007633588
p4 Precision: 0.7247081712062257
p4 Recall: 0.7240038872691934
p4 F1 Score: 0.7243558580456976


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2'],
    'class_weight': ['balanced', None]
}

grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring='f1_macro')
grid.fit(X_train, y_train)

best_rf = grid.best_estimator_

In [ ]:
best_rf

RandomForestClassifier(class_weight='balanced', max_depth=20,
                       min_samples_leaf=2, min_samples_split=5,
                       n_estimators=200, random_state=42)

In [ ]:
rf_classifier= RandomForestClassifier(class_weight='balanced', max_depth=20,
                       min_samples_leaf=2, min_samples_split=5,
                       n_estimators=200, random_state=42)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=20,
                       min_samples_leaf=2, min_samples_split=5,
                       n_estimators=200, random_state=42)

In [ ]:
y_pred = rf_classifier.predict(X_test)

In [ ]:
accuracy=accuracy_score(y_test,y_pred)
print(accuracy)

0.7497919885890884


In [ ]:
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

In [ ]:
for i, v in enumerate(['p1','p2','p3','p4']):
    print(f"{v} Precision: {precision[i]}")
    print(f"{v} Recall: {recall[i]}")
    print(f"{v} F1 Score: {f1_score[i]}")

p1 Precision: 0.7645403377110694
p1 Recall: 0.8037475345167653
p1 F1 Score: 0.7836538461538461
p2 Precision: 0.8512061625785526
p2 Recall: 0.8323092170465808
p2 F1 Score: 0.8416516335939066
p3 Precision: 0.4112587959343237
p3 Recall: 0.3969811320754717
p3 F1 Score: 0.4039938556067588
p4 Precision: 0.6766519823788546
p4 Recall: 0.7463556851311953
p4 F1 Score: 0.7097966728280961


In [ ]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33651 entries, 29847 to 15795
Data columns (total 54 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Age_Newest_TL                 33651 non-null  int64  
 1   pct_PL_enq_L6m_of_ever        33651 non-null  float64
 2   CC_TL                         33651 non-null  int64  
 3   Age_Oldest_TL                 33651 non-null  int64  
 4   GL_Flag                       33651 non-null  int64  
 5   Home_TL                       33651 non-null  int64  
 6   EDUCATION                     33651 non-null  int64  
 7   PL_TL                         33651 non-null  int64  
 8   time_since_recent_payment     33651 non-null  int64  
 9   recent_level_of_deliq         33651 non-null  int64  
 10  num_sub                       33651 non-null  int64  
 11  Secured_TL                    33651 non-null  int64  
 12  pct_CC_enq_L6m_of_ever        33651 non-null  float64
 13  nu

In [ ]:
xgb_classifier = xgb.XGBClassifier(objective='multi:softmax', num_class=4)
xgb_classifier.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=4,
              num_parallel_tree=None, ...)

In [ ]:
y_pred = xgb_classifier.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7751099488886247


In [ ]:
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

In [ ]:
for i, v in enumerate(['p1','p2','p3','p4']):
    print(f"{v} Precision: {precision[i]}")
    print(f"{v} Recall: {recall[i]}")
    print(f"{v} F1 Score: {f1_score[i]}")

p1 Precision: 0.827438370846731
p1 Recall: 0.7613412228796844
p1 F1 Score: 0.79301489470981
p2 Precision: 0.823329760628796
p2 Recall: 0.913577799801784
p2 F1 Score: 0.8661091797425537
p3 Precision: 0.45720984759671746
p3 Recall: 0.2943396226415094
p3 F1 Score: 0.3581267217630854
p4 Precision: 0.7288629737609329
p4 Recall: 0.7288629737609329
p4 F1 Score: 0.7288629737609329


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

xgb_model = XGBClassifier(
    objective='multi:softprob',
    num_class=4,
    eval_metric='mlogloss',
    use_label_encoder=False,
    n_jobs=-1,
    random_state=42
)

param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.3, 0.5],
    'min_child_weight': [1, 3, 5],
    'reg_alpha': [0, 0.01, 0.1],
    'reg_lambda': [1, 1.5, 2],
}

random_search = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_dist,
    n_iter=50,  # Increase to 100+ for deeper tuning
    cv=3,
    scoring='f1_macro',  # Macro to equally weigh each class
    verbose=2,
    random_state=42
)

random_search.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:57:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=0.8, gamma=0, learning_rate=0.01, max_depth=5, min_child_weight=1, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=0.6; total time=   5.3s


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:57:33] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=0.8, gamma=0, learning_rate=0.01, max_depth=5, min_child_weight=1, n_estimators=100, reg_alpha=0, reg_lambda=1, subsample=0.6; total time=   2.4s


...

[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.1, max_depth=7, min_child_weight=5, n_estimators=100, reg_alpha=0.1, reg_lambda=1, subsample=1.0; total time=   2.8s


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:14:13] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.1, max_depth=7, min_child_weight=5, n_estimators=100, reg_alpha=0.1, reg_lambda=1, subsample=1.0; total time=   2.9s


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:14:16] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='mlogloss',
                                           feature_types=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learnin...
                                           num_class=4, num_parallel_tree=None, ...),
                   n_iter=50,
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                                        'gamma': [0, 0.1, 0.3, 0.5],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                        'max_depth': [3, 5, 7, 9],
                                        'min_child_weight': [1, 3, 5],
                                        'n_estimators': [100, 200, 300],
                                        'reg_alpha': [0, 0.01, 0.1],
                                        'reg_lambda': [1, 1.5, 2],
                                        'subsample': [0.6, 0.8, 1.0]},
                   random_state=42, scoring='f1_macro', verbose=2)

In [ ]:
best_xgb = random_search.best_estimator_

y_pred = best_xgb.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=['p1','p2','p3','p4']))

              precision    recall  f1-score   support

          p1       0.83      0.76      0.79      1014
          p2       0.82      0.92      0.87      5045
          p3       0.47      0.29      0.36      1325
          p4       0.73      0.74      0.73      1029

    accuracy                           0.78      8413
   macro avg       0.71      0.68      0.69      8413
weighted avg       0.76      0.78      0.76      8413



In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# First, do your original train-test split
X_train, X_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, stratify=y, random_state=42)

# Then apply SMOTE only to the training data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)


In [ ]:
# Convert the NumPy array y_resampled back to a pandas Series
y_resampled = pd.Series(y_resampled)

# Now you can call value_counts() on the pandas Series
y_resampled.value_counts()

,count
3,20362
2,20362
0,20362
1,20362


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

xgb = XGBClassifier(
    objective='multi:softmax',
    num_class=4,
    learning_rate=0.05,
    n_estimators=300,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=3,
    gamma=0.3,
    use_label_encoder=False,
    eval_metric='mlogloss',
    random_state=42
)

xgb.fit(X_resampled, y_resampled)
y_pred = xgb.predict(X_test)

# Evaluate
print(classification_report(y_test, y_pred, target_names=['p1', 'p2', 'p3', 'p4']))

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [19:35:04] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

          p1       0.77      0.83      0.80       982
          p2       0.83      0.88      0.85      5090
          p3       0.41      0.27      0.33      1288
          p4       0.73      0.75      0.74      1053

    accuracy                           0.77      8413
   macro avg       0.69      0.68      0.68      8413
weighted avg       0.75      0.77      0.75      8413

